In [ ]:
### Create points from likely intersections ###

import fiona
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import mapping

# Open the multipolygon shapefile with likely intersections
with fiona.open('') as src:
    meta = src.meta
    schema = {
    'geometry': 'Point',
    'properties': {'id': 'int'},
    }
    meta['schema'] = schema
    # Create output point shapefile
    with fiona.open('', 'w', **meta) as dst:
        for f in src:
            geom = f['geometry']
            my_shape = shape(geom)
            if geom['type'] == 'MultiPolygon':
                c = 0
                for temp in my_shape:
                    centroid = Polygon(temp).centroid
                    dst.write({'geometry': mapping(centroid), 'properties': {'id': 'int'},})
                    c = c + 1
            else:
                print('Not a multipolygon')

In [ ]:
### Calculate the number of intersections and road area per cell for a polygon grid file ###

import fiona
from shapely.geometry import shape, mapping
from rtree import index

def calc_connectivity(grid, intersections, classified, attribute, outname):    
    road_idx = index.Index()
    with fiona.collection(classified, 'r') as polygon:
        polygons = [elem for elem in polygon]
        for poly in polygons:
            if poly['properties']['predicted'] == 2:
                fid = int(poly['id'])
                print(fid)
                geom = shape(poly['geometry'])
                if geom.is_valid == False:
                    geom = geom.buffer(0)
                if geom.is_valid:
                    road_idx.insert(fid, geom.bounds)
                else:
                    print('Warning: Invalid geometry')
    intersection_idx = index.Index()
    with fiona.collection(intersections, 'r') as inter:
        points = [elem for elem in inter]
        for point in points:
            inter_fid = int(point['id'])
            p = shape(point['geometry'])
            intersection_idx.insert(inter_fid, p.bounds)
    with fiona.collection(grid, 'r') as poly_grid:
        cells = [elem for elem in poly_grid]
        meta = poly_grid.meta
        #meta['schema']['geometry'] = 'Polygon'
        meta['schema']['properties']['no_intersections'] = 'int'
        meta['schema']['properties']['road_area'] = 'float'
        cell_area = shape(cells[0]['geometry']).area
        if cell_area == 0:
             raise ValueError('Grid is not valid.')
        with fiona.open(outname, 'w', **meta) as dst:
            cc = 0
            for cell in cells:
                cell_geom = shape(cell['geometry'])
                count_inter = 0
                road_area = 0
                for int_id in list(intersection_idx.intersection(cell_geom.bounds)):
                    if shape(points[int_id]['geometry']).within(cell_geom):
                        count_inter = count_inter + 1
                for r_id in list(road_idx.intersection(cell_geom.bounds)):
                    sh_poly = shape(polygons[r_id]['geometry'])
                    if sh_poly.is_valid == False:
                        sh_poly = sh_poly.buffer(0)
                    if sh_poly.intersects(cell_geom):
                        in_cell = sh_poly.intersection(cell_geom)
                        area_in_cell = in_cell.area
                        road_area = road_area + area_in_cell
                res = {}
                res['properties'] = cell['properties']
                res['properties']['no_intersections'] = count_inter
                res['properties']['road_area'] = road_area / cell_area * 100
                res['geometry'] = mapping(shape(cell['geometry']))
                dst.write(res)
                print('added cell ' + str(cc) + ' of ' + str(len(cells)))
                cc = cc + 1
                        

if __name__ == '__main__':
    # Likely intersections as point shapefile (output of the cell above)
    inter_points = ''
    # A polygon grid, result of script 00_Create_grid
    gridfile = ''
    # Shapefile with the classification; use road polygon shapefile and adapt to save computation time
    classi = ''
    # Output file
    out = ''
    # 'predict
    calc_connectivity(gridfile, inter_points, classi, 'predicted', out)